<a href="https://colab.research.google.com/github/nyp-sit/sdaai-pdc2-students/blob/master/iti107/session-5/od_using_tfod_api/object_detection_tfod_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" align="left"/></a>

# Object Detection using custom trained model

Now that you have trained a custom object detection model, it is time to test it out! 

The codes below are similar to the previous exercise you have done using the pretrained model on COCO dataset. 
Mofify the codes below to use your 'raccoon detector'.

## 1. Imports

In [ ]:
import numpy as np
import os
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

## 2. Model preparation 

### Load the saved model 

Here we setup the model directory to point to the exported model directory and load the model as usual using ``tf.saved_model.load()``.

In [ ]:
project_root_dir = '/home/ubuntu/raccoon_project'
model_dir = os.path.join(project_root_dir, "models/export_model/saved_model")

# load the model
detection_model = tf.saved_model.load(str(model_dir))


### Label Map file

Now let us create a dictionary of the label to class mapping using the label_map_util as follow.

In [ ]:
PATH_TO_LABELS = "/home/ubuntu/raccoon_project/data/raccoon_label_map.pbtxt"
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# let's print out a few entries to see what are the different objects we have
print(category_index)

## 3. Object Detection 

The code below is the same as the ones used in the exercise on Objection Detection using TFOD.

In [ ]:
def run_inference_for_single_image(model, image_np):
    
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    #model_fn = model.signatures['serving_default']
    output_dict = model(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

    vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          min_score_thresh=.5,
          use_normalized_coordinates=True,
          line_thickness=8)
    
    # return image that has been drawn on
    return image_np



### Perform detection on a test Image

We read in the test file ``sample_raccoon.jpg`` file and run inference on it using the custom model we trained.

In [ ]:
image_path = 'sample_raccoon.jpg'
image_np = np.array(Image.open(image_path))
# Actual detection.
processed_image = run_inference_for_single_image(detection_model, image_np)
display(Image.fromarray(image_np))
